In [ ]:
# raw data on April 1, 2022 from 16:10 to 16:40 UTC. Launch from SLC40 at 16:24 UTC, watched from Titusville
import os, sys, glob, obspy
import numpy as np
import pandas as pd
cwd = os.getcwd()
sys.path.append(os.path.join(cwd, 'campbell'))
import read_cs_files as campbell

HOME = os.getenv('HOME')
SDS_TOP = os.path.join(HOME, 'Desktop/Desktop - Glenn’s MacBook Air/KSC_new')
DROPBOX_TOP = os.path.join(HOME, 'Dropbox')
DROPBOX_PROJECT_DIR = os.path.join(DROPBOX_TOP, 'PROFESSIONAL/RESEARCH/3 Project Documents/NASAprojects/202010 KSC Launchpad Erosion')
EVENT_MSEED_DIR = os.path.join(DROPBOX_PROJECT_DIR, 'KSC_Well_Seismoacoustic_Data/SeismoAcousticData/Events')
HTML_DIR = './html'
PLOTS_DIR = HTML_DIR

WELLDATA_TOP = os.path.join(DROPBOX_PROJECT_DIR, 'KSC_Well_Seismoacoustic_Data/WellData')
TOB3_DIR = os.path.join(WELLDATA_TOP, 'Uploads')
WELLMSEED_DIR = os.path.join(WELLDATA_TOP, 'miniseed')


def localtime2utc(this_dt):
    localTimeCorrection = 3600 * 4
    return this_dt + localTimeCorrection

# Read Campbell Scientific TOB3 file
allcolumns = []
tob3files = glob.glob(os.path.join(TOB3_DIR, '2022????/*.dat'))
lod = []
for tob3file in tob3files:
    data, meta = campbell.read_cs_files(tob3file, forcedatetime=False,
                      bycol=True, quiet=True, metaonly=False)

    df = pd.DataFrame(columns=meta[2])
    for c in range(len(meta[2])):
        df[meta[2][c]] = data[c]
        if not meta[2][c] in allcolumns:
            allcolumns.append(meta[2][c])
          
    # convert to Miniseed
    local_startt = obspy.UTCDateTime(df.iloc[0]['TIMESTAMP'])
    nextt = obspy.UTCDateTime(df.iloc[1]['TIMESTAMP'])
    dt = nextt-local_startt
    utc_startt = localtime2utc(local_startt)

    tob3base = os.path.basename(tob3file)
    if tob3base[:3]=='HOF':
        sta = tob3base[9:11]
    elif tob3base[:8]=='Absolute':
        sta = 'AB'
    
    for this_col in df.columns[2:]:
        tr = obspy.Trace()
        tr.id = 'FL.S39%s.00.' % sta
        if this_col[1]=='_' and this_col[-1]=='A':
            tr.stats.station = 'AB6%s' % this_col[0] 
        if 'TempC' in this_col:
            tr.stats.channel = 'HKD'
        elif this_col=='Baro':
            tr.stats.channel = 'HDH'
        elif 'psi' in this_col:
            tr.stats.channel = 'HDD'
            if this_col[1]=='_' and this_col[-1]=='A':
                tr.stats.location = this_col[2:4] 
            else:
                tr.stats.location = this_col[:2]
        tr.stats.starttime = utc_startt
        tr.stats.delta = dt
        mseedfile = os.path.join(WELLMSEED_DIR, '%s.%s' % (utc_startt.strftime('%Y%m%d%H%M%S'), tr.id  ) )
        tr.data = np.array(df[this_col])
        print('Writing %s' % mseedfile)
        tr.write(mseedfile, 'mseed')
        thisrow = {'TOB3file':tob3base, 'miniseedFile':os.path.basename(mseedfile)}
        lod.append(thisrow)
df = pd.DataFrame(lod)
df.to_csv(os.path.join(WELLDATA_TOP,'lookuptable_tob3file_miniseedfile.csv'))

Writing /Users/thompsong/Dropbox/PROFESSIONAL/RESEARCH/3 Project Documents/NASAprojects/202010 KSC Launchpad Erosion/KSC_Well_Seismoacoustic_Data/WellData/miniseed/20220421200000.FL.AB6I.30.HDD
Writing /Users/thompsong/Dropbox/PROFESSIONAL/RESEARCH/3 Project Documents/NASAprojects/202010 KSC Launchpad Erosion/KSC_Well_Seismoacoustic_Data/WellData/miniseed/20220421200000.FL.AB6S.30.HDD
Writing /Users/thompsong/Dropbox/PROFESSIONAL/RESEARCH/3 Project Documents/NASAprojects/202010 KSC Launchpad Erosion/KSC_Well_Seismoacoustic_Data/WellData/miniseed/20220531160000.FL.AB6I.30.HDD
Writing /Users/thompsong/Dropbox/PROFESSIONAL/RESEARCH/3 Project Documents/NASAprojects/202010 KSC Launchpad Erosion/KSC_Well_Seismoacoustic_Data/WellData/miniseed/20220531160000.FL.AB6S.30.HDD
Writing /Users/thompsong/Dropbox/PROFESSIONAL/RESEARCH/3 Project Documents/NASAprojects/202010 KSC Launchpad Erosion/KSC_Well_Seismoacoustic_Data/WellData/miniseed/20220603000000.FL.AB6I.30.HDD
Writing /Users/thompsong/Dropb

Writing /Users/thompsong/Dropbox/PROFESSIONAL/RESEARCH/3 Project Documents/NASAprojects/202010 KSC Launchpad Erosion/KSC_Well_Seismoacoustic_Data/WellData/miniseed/20220425080000.FL.S396I.00.HKD
Writing /Users/thompsong/Dropbox/PROFESSIONAL/RESEARCH/3 Project Documents/NASAprojects/202010 KSC Launchpad Erosion/KSC_Well_Seismoacoustic_Data/WellData/miniseed/20220509080000.FL.AB6I.30.HDD
Writing /Users/thompsong/Dropbox/PROFESSIONAL/RESEARCH/3 Project Documents/NASAprojects/202010 KSC Launchpad Erosion/KSC_Well_Seismoacoustic_Data/WellData/miniseed/20220509080000.FL.AB6S.30.HDD
Writing /Users/thompsong/Dropbox/PROFESSIONAL/RESEARCH/3 Project Documents/NASAprojects/202010 KSC Launchpad Erosion/KSC_Well_Seismoacoustic_Data/WellData/miniseed/20220513160000.FL.AB6I.30.HDD
Writing /Users/thompsong/Dropbox/PROFESSIONAL/RESEARCH/3 Project Documents/NASAprojects/202010 KSC Launchpad Erosion/KSC_Well_Seismoacoustic_Data/WellData/miniseed/20220513160000.FL.AB6S.30.HDD
Writing /Users/thompsong/Drop

Writing /Users/thompsong/Dropbox/PROFESSIONAL/RESEARCH/3 Project Documents/NASAprojects/202010 KSC Launchpad Erosion/KSC_Well_Seismoacoustic_Data/WellData/miniseed/20220505160000.FL.S396S.00.HKD
Writing /Users/thompsong/Dropbox/PROFESSIONAL/RESEARCH/3 Project Documents/NASAprojects/202010 KSC Launchpad Erosion/KSC_Well_Seismoacoustic_Data/WellData/miniseed/20220516080000.FL.S396I.00.HDH
Writing /Users/thompsong/Dropbox/PROFESSIONAL/RESEARCH/3 Project Documents/NASAprojects/202010 KSC Launchpad Erosion/KSC_Well_Seismoacoustic_Data/WellData/miniseed/20220516080000.FL.S396I.15.HDD
Writing /Users/thompsong/Dropbox/PROFESSIONAL/RESEARCH/3 Project Documents/NASAprojects/202010 KSC Launchpad Erosion/KSC_Well_Seismoacoustic_Data/WellData/miniseed/20220516080000.FL.S396I.70.HDD
Writing /Users/thompsong/Dropbox/PROFESSIONAL/RESEARCH/3 Project Documents/NASAprojects/202010 KSC Launchpad Erosion/KSC_Well_Seismoacoustic_Data/WellData/miniseed/20220516080000.FL.S396I.30.HDD
Writing /Users/thompsong/

Writing /Users/thompsong/Dropbox/PROFESSIONAL/RESEARCH/3 Project Documents/NASAprojects/202010 KSC Launchpad Erosion/KSC_Well_Seismoacoustic_Data/WellData/miniseed/20220506120000.FL.S396S.00.HKD
Writing /Users/thompsong/Dropbox/PROFESSIONAL/RESEARCH/3 Project Documents/NASAprojects/202010 KSC Launchpad Erosion/KSC_Well_Seismoacoustic_Data/WellData/miniseed/20220504080000.FL.S396S.00.HDH
Writing /Users/thompsong/Dropbox/PROFESSIONAL/RESEARCH/3 Project Documents/NASAprojects/202010 KSC Launchpad Erosion/KSC_Well_Seismoacoustic_Data/WellData/miniseed/20220504080000.FL.S396S.15.HDD
Writing /Users/thompsong/Dropbox/PROFESSIONAL/RESEARCH/3 Project Documents/NASAprojects/202010 KSC Launchpad Erosion/KSC_Well_Seismoacoustic_Data/WellData/miniseed/20220504080000.FL.S396S.70.HDD
Writing /Users/thompsong/Dropbox/PROFESSIONAL/RESEARCH/3 Project Documents/NASAprojects/202010 KSC Launchpad Erosion/KSC_Well_Seismoacoustic_Data/WellData/miniseed/20220504080000.FL.S396S.30.HDD
Writing /Users/thompsong/

Writing /Users/thompsong/Dropbox/PROFESSIONAL/RESEARCH/3 Project Documents/NASAprojects/202010 KSC Launchpad Erosion/KSC_Well_Seismoacoustic_Data/WellData/miniseed/20220428080000.FL.AB6I.30.HDD
Writing /Users/thompsong/Dropbox/PROFESSIONAL/RESEARCH/3 Project Documents/NASAprojects/202010 KSC Launchpad Erosion/KSC_Well_Seismoacoustic_Data/WellData/miniseed/20220428080000.FL.AB6S.30.HDD
Writing /Users/thompsong/Dropbox/PROFESSIONAL/RESEARCH/3 Project Documents/NASAprojects/202010 KSC Launchpad Erosion/KSC_Well_Seismoacoustic_Data/WellData/miniseed/20220517000000.FL.S396S.00.HDH
Writing /Users/thompsong/Dropbox/PROFESSIONAL/RESEARCH/3 Project Documents/NASAprojects/202010 KSC Launchpad Erosion/KSC_Well_Seismoacoustic_Data/WellData/miniseed/20220517000000.FL.S396S.15.HDD
Writing /Users/thompsong/Dropbox/PROFESSIONAL/RESEARCH/3 Project Documents/NASAprojects/202010 KSC Launchpad Erosion/KSC_Well_Seismoacoustic_Data/WellData/miniseed/20220517000000.FL.S396S.70.HDD
Writing /Users/thompsong/Dr

...

Writing /Users/thompsong/Dropbox/PROFESSIONAL/RESEARCH/3 Project Documents/NASAprojects/202010 KSC Launchpad Erosion/KSC_Well_Seismoacoustic_Data/WellData/miniseed/20220511080000.FL.S396I.00.HKD
Writing /Users/thompsong/Dropbox/PROFESSIONAL/RESEARCH/3 Project Documents/NASAprojects/202010 KSC Launchpad Erosion/KSC_Well_Seismoacoustic_Data/WellData/miniseed/20220513160000.FL.S396I.00.HDH
Writing /Users/thompsong/Dropbox/PROFESSIONAL/RESEARCH/3 Project Documents/NASAprojects/202010 KSC Launchpad Erosion/KSC_Well_Seismoacoustic_Data/WellData/miniseed/20220513160000.FL.S396I.15.HDD
Writing /Users/thompsong/Dropbox/PROFESSIONAL/RESEARCH/3 Project Documents/NASAprojects/202010 KSC Launchpad Erosion/KSC_Well_Seismoacoustic_Data/WellData/miniseed/20220513160000.FL.S396I.70.HDD
Writing /Users/thompsong/Dropbox/PROFESSIONAL/RESEARCH/3 Project Documents/NASAprojects/202010 KSC Launchpad Erosion/KSC_Well_Seismoacoustic_Data/WellData/miniseed/20220513160000.FL.S396I.30.HDD
Writing /Users/thompsong/

Writing /Users/thompsong/Dropbox/PROFESSIONAL/RESEARCH/3 Project Documents/NASAprojects/202010 KSC Launchpad Erosion/KSC_Well_Seismoacoustic_Data/WellData/miniseed/20220531160000.FL.S396I.30.HDD
Writing /Users/thompsong/Dropbox/PROFESSIONAL/RESEARCH/3 Project Documents/NASAprojects/202010 KSC Launchpad Erosion/KSC_Well_Seismoacoustic_Data/WellData/miniseed/20220531160000.FL.S396I.00.HKD
Writing /Users/thompsong/Dropbox/PROFESSIONAL/RESEARCH/3 Project Documents/NASAprojects/202010 KSC Launchpad Erosion/KSC_Well_Seismoacoustic_Data/WellData/miniseed/20220428000000.FL.S396I.00.HDH
Writing /Users/thompsong/Dropbox/PROFESSIONAL/RESEARCH/3 Project Documents/NASAprojects/202010 KSC Launchpad Erosion/KSC_Well_Seismoacoustic_Data/WellData/miniseed/20220428000000.FL.S396I.15.HDD
Writing /Users/thompsong/Dropbox/PROFESSIONAL/RESEARCH/3 Project Documents/NASAprojects/202010 KSC Launchpad Erosion/KSC_Well_Seismoacoustic_Data/WellData/miniseed/20220428000000.FL.S396I.70.HDD
Writing /Users/thompsong/

Writing /Users/thompsong/Dropbox/PROFESSIONAL/RESEARCH/3 Project Documents/NASAprojects/202010 KSC Launchpad Erosion/KSC_Well_Seismoacoustic_Data/WellData/miniseed/20220430160000.FL.S396I.00.HKD
Writing /Users/thompsong/Dropbox/PROFESSIONAL/RESEARCH/3 Project Documents/NASAprojects/202010 KSC Launchpad Erosion/KSC_Well_Seismoacoustic_Data/WellData/miniseed/20220424120000.FL.AB6I.30.HDD
Writing /Users/thompsong/Dropbox/PROFESSIONAL/RESEARCH/3 Project Documents/NASAprojects/202010 KSC Launchpad Erosion/KSC_Well_Seismoacoustic_Data/WellData/miniseed/20220424120000.FL.AB6S.30.HDD
Writing /Users/thompsong/Dropbox/PROFESSIONAL/RESEARCH/3 Project Documents/NASAprojects/202010 KSC Launchpad Erosion/KSC_Well_Seismoacoustic_Data/WellData/miniseed/20220425120000.FL.AB6I.30.HDD
Writing /Users/thompsong/Dropbox/PROFESSIONAL/RESEARCH/3 Project Documents/NASAprojects/202010 KSC Launchpad Erosion/KSC_Well_Seismoacoustic_Data/WellData/miniseed/20220425120000.FL.AB6S.30.HDD
Writing /Users/thompsong/Drop

Writing /Users/thompsong/Dropbox/PROFESSIONAL/RESEARCH/3 Project Documents/NASAprojects/202010 KSC Launchpad Erosion/KSC_Well_Seismoacoustic_Data/WellData/miniseed/20220531200000.FL.S396S.30.HDD
Writing /Users/thompsong/Dropbox/PROFESSIONAL/RESEARCH/3 Project Documents/NASAprojects/202010 KSC Launchpad Erosion/KSC_Well_Seismoacoustic_Data/WellData/miniseed/20220531200000.FL.S396S.00.HKD
Writing /Users/thompsong/Dropbox/PROFESSIONAL/RESEARCH/3 Project Documents/NASAprojects/202010 KSC Launchpad Erosion/KSC_Well_Seismoacoustic_Data/WellData/miniseed/20220601200000.FL.S396S.00.HDH
Writing /Users/thompsong/Dropbox/PROFESSIONAL/RESEARCH/3 Project Documents/NASAprojects/202010 KSC Launchpad Erosion/KSC_Well_Seismoacoustic_Data/WellData/miniseed/20220601200000.FL.S396S.15.HDD
Writing /Users/thompsong/Dropbox/PROFESSIONAL/RESEARCH/3 Project Documents/NASAprojects/202010 KSC Launchpad Erosion/KSC_Well_Seismoacoustic_Data/WellData/miniseed/20220601200000.FL.S396S.70.HDD
Writing /Users/thompsong/

Writing /Users/thompsong/Dropbox/PROFESSIONAL/RESEARCH/3 Project Documents/NASAprojects/202010 KSC Launchpad Erosion/KSC_Well_Seismoacoustic_Data/WellData/miniseed/20220531120000.FL.S396I.30.HDD
Writing /Users/thompsong/Dropbox/PROFESSIONAL/RESEARCH/3 Project Documents/NASAprojects/202010 KSC Launchpad Erosion/KSC_Well_Seismoacoustic_Data/WellData/miniseed/20220531120000.FL.S396I.00.HKD
Writing /Users/thompsong/Dropbox/PROFESSIONAL/RESEARCH/3 Project Documents/NASAprojects/202010 KSC Launchpad Erosion/KSC_Well_Seismoacoustic_Data/WellData/miniseed/20220426080000.FL.AB6I.30.HDD
Writing /Users/thompsong/Dropbox/PROFESSIONAL/RESEARCH/3 Project Documents/NASAprojects/202010 KSC Launchpad Erosion/KSC_Well_Seismoacoustic_Data/WellData/miniseed/20220426080000.FL.AB6S.30.HDD
Writing /Users/thompsong/Dropbox/PROFESSIONAL/RESEARCH/3 Project Documents/NASAprojects/202010 KSC Launchpad Erosion/KSC_Well_Seismoacoustic_Data/WellData/miniseed/20220512080000.FL.S396S.00.HDH
Writing /Users/thompsong/Dr